In [0]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Reshape, Input, BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np
import sys

In [0]:
latent_dim = 100
nh = 28
nw = 28
nc = 1
img_shape = (nh, nw, nc)

opt = Adam(0.0002, 0.5)

In [0]:
def build_generator():
  
  model = Sequential()
  
  model.add(Dense(128 * 7 * 7, activation="relu", input_dim = latent_dim))
  model.add(Reshape((7, 7, 128)))
  
  #deconvolve step, upsampling will increase the dimension of input image by repeating rows and columns
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(Conv2D(nc, kernel_size=3, padding="same"))
  model.add(Activation("tanh"))
  
  print(model.summary())
  
  noise = Input(shape = (latent_dim,))
  img = model(noise)
  
  return Model(inputs = noise, outputs = img)
  

In [23]:
generator = build_generator()
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 28, 28, 128)       0         
__________

In [0]:
def build_discriminator():
  model = Sequential()
  
  model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = img_shape, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(ZeroPadding2D(padding = ((0,1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))
  
  print(model.summary())
  
  img = Input(shape = img_shape)
  validity = model(img)
  
  return Model(inputs = img, outputs = validity)

In [25]:
discriminator = build_discriminator()
discriminator.summary()

discriminator.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

In [26]:
z = Input(shape = (latent_dim,))
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss = 'binary_crossentropy', optimizer = opt)
combined.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_4 (Model)              (None, 28, 28, 1)         856193    
_________________________________________________________________
model_5 (Model)              (None, 1)                 390657    
Total params: 1,246,850
Trainable params: 855,809
Non-trainable params: 391,041
_________________________________________________________________


In [27]:
print(generator.summary())
print(discriminator.summary())
print(combined.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 28, 28, 1)         856193    
Total params: 856,193
Trainable params: 855,809
Non-trainable params: 384
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 390657    
Total params: 780,418
Trainable params: 389,761
Non-trainable params: 390,657
_________________________________________________________________
None
______________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [0]:
def train(batch_size = 128, epochs = 4000):
  
  (X_train, _), (_, _) = mnist.load_data()

  X_train = X_train / 127.5 - 1.
  X_train = np.expand_dims(X_train, axis=3)
  print(X_train.shape)

  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))
  
  for epoch in range(epochs):
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)
    
    #Training Discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5*np.add(d_loss_real, d_loss_fake)
    
    #Training Generator
    g_loss = combined.train_on_batch(noise, valid)
    
    #Progress
    print("epoch: " + str(epoch) + " " + "D_Loss = " + str(d_loss[0]) + " " + "acc: " + str(d_loss[1]*100) + " " +  "G_Loss = " + str(g_loss))

In [10]:
train(128, 4000)

11493376/11490434 [==============================] - 2s 0us/step
(60000, 28, 28, 1)
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0 D_Loss = 1.0234721 acc: 40.625 G_Loss = 1.0425559
epoch: 1 D_Loss = 0.63855845 acc: 66.796875 G_Loss = 0.9721089
epoch: 2 D_Loss = 0.5342928 acc: 73.828125 G_Loss = 0.88261473
epoch: 3 D_Loss = 0.41668373 acc: 81.25 G_Loss = 0.7383447
epoch: 4 D_Loss = 0.3221869 acc: 88.28125 G_Loss = 0.661826
epoch: 5 D_Loss = 0.26324132 acc: 92.96875 G_Loss = 0.62140566
epoch: 6 D_Loss = 0.37795877 acc: 83.59375 G_Loss = 0.8131864
epoch: 7 D_Loss = 0.78425026 acc: 57.8125 G_Loss = 1.2556349
epoch: 8 D_Loss = 1.0300863 acc: 42.96875 G_Loss = 1.8153092
epoch: 9 D_Loss = 0.9353634 acc: 47.65625 G_Loss = 1.4686319
epoch: 10 D_Loss = 0.6907922 acc: 58.59375 G_Loss = 1.3094819
epoch: 11 D_Loss = 0.7282764 acc: 59.375 G_Loss = 1.1570395
epoch: 12 D_Loss = 0.8964325 acc: 50.390625 G_Loss = 1.1521444
epoch: 13 D_Loss = 0.6996521 acc: 60.15625 G_Loss = 0.93174005
epoch: 14 D_Loss = 0.5206539 acc: 71.875 G_Loss = 0.7701488
epoch: 15 D_Loss = 0.40793458 acc: 82.03125 G_Loss = 0.6293115
epoch: 16 D_Loss 

In [0]:
def save_imgs(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
     for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
  fig.savefig("images/mnist_%d.png" % epoch)
  plt.close()

In [0]:
save_imgs(4000)